In [26]:
import numpy as np
import cirq

In [27]:
def main():
    circuit = make_bell_test_circuit()
    print("Circuit:")
    print(circuit)
    
    #Run Simulation
    sim = cirq.Simulator()
    repetitions = 1000
    print("Simulating {} repetitions".format(repetitions))
    result = sim.run(program=circuit, repetitions=repetitions)
    
    ## Collect Results

    a = np.array(result.measurements['a'] [:, 0])
    b = np.array(result.measurements['b'] [:, 0])
    x = np.array(result.measurements['x'] [:, 0])
    y = np.array(result.measurements['y'] [:, 0])
    
    outcomes = a ^ b == x & y
    win_percent = len([e for e in outcomes if e]) * 100/repetitions
    
    print()
    print('Results')
    print('a:', bitstring(a))
    print('b:', bitstring(b))
    print('x:', bitstring(x))
    print('y:', bitstring(y))
    print('(a XOR b) == (x AND y): \n', bitstring(outcomes))
    print('Win rate:  {}%' .format(win_percent))

In [28]:
def make_bell_test_circuit():
    alice = cirq.GridQubit(0, 0)
    bob = cirq.GridQubit(1, 0)
    alice_referee = cirq.GridQubit(0, 1)
    bob_referee = cirq.GridQubit(1,1)
    
    circuit = cirq.Circuit()
    
    ## Prepare shared entanglement 
    circuit.append([cirq.H(alice),
                  cirq.CNOT(alice, bob),
                    cirq.X(alice)**-0.25,])
    
    ## Referee flip coins
    circuit.append([cirq.H(alice_referee),
                   cirq.H(bob_referee),
                   ])
    
    ## Players do oa sqrt based on their referee coin
    circuit.append([cirq.CNOT(alice_referee, alice)**0.5,
                   cirq.CNOT(bob_referee, bob)**0.5,
                   ])
    
    circuit.append([
        cirq.measure(alice, key = 'a'),
        cirq.measure(bob, key = 'b'),
        cirq.measure(alice_referee, key = 'x'),
        cirq.measure(bob_referee, key = 'y'),
    ])
    
    return circuit


In [29]:
def bitstring(bits):
    return ''.join('1' if e else '_' for e in bits)

In [30]:
if __name__ == '__main__':
    main()

Circuit:
(0, 0): ───H───@───X^-0.25───X────────M('a')───
               │             │
(0, 1): ───H───┼─────────────@^0.5────M('x')───
               │
(1, 0): ───────X───X─────────M('b')────────────
                   │
(1, 1): ───H───────@^0.5─────M('y')────────────
Simulating 1000 repetitions

Results
a: ______11_11_______1_1_1_11_1111__1__1_11_1____111_1_1__1_1_1_1_11_1__11______1___1_____11_1_1_1___11_111__1____1_11____1_11_1_1_1__111__1111111___111_11_11__111____1111_111__11_1111___11__11_1111_11_____11_11111__1_1_11__1111_11_1_1111_1__1_11111_1_111__1__1_1_1__11____111___1_111_1_11____11111___1_111_1_1_11__11____1_1____1____1_11____111___111___11___1__111__1_________1___11__1___1_1_____11_1____1_1__1____1111_11111_1_1_11_111__1_1__1__111________1__1111_11111111__111_1_11_1___1___111_11111_11__1_1__111__1__1_11__111__1__11_1__1___1_111___111_1___111___1111_1_1__111__11_1111__111_11_____11_1111_1_11_1_11_11_1__1__11___11_____111____1_111___1__1_____1_11111_11_111______11___111_1_